\begin{aligned}
\\
\\
\LARGE\text{Multi-factor Momentum Strategy (name is tentative)} \\
\\
\end{aligned}

$$ \text{[FINM 33150] Regression Analysis and Quantitative Trading Strategies} $$
$$ \text{Winter 2022 | Professor Brian Boonstra} $$
$$ \text{Final Project Draft} $$

\begin{aligned}
\small\text{Authors: Ashley Tsoi}^1\small\text{, Pi}&\small\text{yush Kontu}^2\small\text{, Gauri Kant}^3
\end{aligned}

$$
\begin{aligned}
^1 \scriptsize\text{atsoi@uchicago.edu }&\scriptsize\text{| Student ID: 12286230} \\
^2 \scriptsize\text{pkontu@uchicago.edu }&\scriptsize\text{| Student ID: 12284351} \\
^3 \scriptsize\text{gkant@uchicago.edu }&\scriptsize\text{| Student ID: 12334466} \\
\\
\end{aligned}
$$

$ 
\textbf{Abstract. }
\text{
    In this paper, we explore a multi-factor time-series momentum
    }
$

$
\textbf{Keywords:}
\text{
    Momentum strategy, covariance, sentiment analysis.
}
$

$\large\textbf{1 }\quad\large\textbf{Introduction}$

$
\text{
    In Generalized Momentum and Flexible Asset Allocation (FAA), keller
    ...
    }
$

$\large\textbf{2 }\quad\large\textbf{Data}$

$\textbf{2 - 1. Define data scope}$

$\text{For the analysis, we selected 20 US ETFs}^4\text{:}$

<br>
<table>
    <tr>
        <td >$\textbf{ETF Symbol}$</td>
        <td>$\textbf{ETF Name}$</td>
        <td>$\textbf{Asset Class}$</td>
        <td>$\textbf{Sub Asset Class}$</td>
    </tr>
    <tr>
        <td>$\text{BIL}$</td>
        <td>$\text{SPDR Bloomberg 1-3 Month T-Bill ETF}$</td>
        <td>$\text{Cash}$</td>
        <td>$\text{Cash}$</td>
    </tr>
    <tr>
        <td>$\text{BNDX}$</td>
        <td>$\text{Vanguard Total International Bond ETF}$</td>
        <td>$\text{Fixed Income}$</td>
        <td>$\text{International Bonds}$</td>
    </tr>
    <tr>
        <td>$\text{COW}$</td>
        <td>$\text{iPath Series B Bloomberg Livestock Subindex Total Return ETN}$</td>
        <td>$\text{Commodities}$</td>
        <td>$\text{Livestock}$</td>
    </tr>
    <tr>
        <td>$\text{DBA}$</td>
        <td>$\text{Invesco DB Agriculture Fund}$</td>
        <td>$\text{Commodities}$</td>
        <td>$\text{Agriculture}$</td>
    </tr>
    <tr>
        <td>$\text{EMB}$</td>
        <td>$\text{iShares JP Morgan USD Emerging Markets Bond ETF}$</td>
        <td>$\text{Fixed Income}$</td>
        <td>$\text{EM Bonds}$</td>
    </tr>
    <tr>
        <td>$\text{FM}$</td>
        <td>$\text{iShares MSCI Frontier 100 ETF}$</td>
        <td>$\text{Equities}$</td>
        <td>$\text{FM-Equities}$</td>
    </tr>
    <tr>
        <td>$\text{GLD}$</td>
        <td>$\text{SPDR Gold Shares}$</td>
        <td>$\text{Commodities}$</td>
        <td>$\text{Gold}$</td>
    </tr>
    <tr>
        <td>$\text{GOVT}$</td>
        <td>$\text{iShares U.S. Treasury Bond ETF}$</td>
        <td>$\text{Fixed Income}$</td>
        <td>$\text{US Treasuries}$</td>
    </tr>
    <tr>
        <td>$\text{HYG}$</td>
        <td>$\text{iShares iBoxx \$ High Yield Corporate Bond ETF}$</td>
        <td>$\text{Fixed Income}$</td>
        <td>$\text{US Corporate High Yield}$</td>
    </tr>
    <tr>
        <td>$\text{ITOT}$</td>
        <td>$\text{iShares Core S&P Total US Stock Market ETF}$</td>
        <td>$\text{Equities}$</td>
        <td>$\text{US-Equities}$</td>
    </tr>
    <tr>
        <td>$\text{LQD}$</td>
        <td>$\text{iShares iBoxx \$ Investment Grade Corporate Bond ETF}$</td>
        <td>$\text{Fixed Income}$</td>
        <td>$\text{US Corporate Bonds}$</td>
    </tr>
    <tr>
        <td>$\text{MBB}$</td>
        <td>$\text{iShares MBS ETF}$</td>
        <td>$\text{Fixed Income}$</td>
        <td>$\text{US MBS}$</td>
    </tr>
    <tr>
        <td>$\text{PSP}$</td>
        <td>$\text{Invesco Global Listed Private Equity ETF}$</td>
        <td>$\text{Private Equity}$</td>
        <td>$\text{Private Equity}$</td>
    </tr>
    <tr>
        <td>$\text{SLV}$</td>
        <td>$\text{iShares Silver Trust}$</td>
        <td>$\text{Commodities}$</td>
        <td>$\text{Silver}$</td>
    </tr>
    <tr>
        <td>$\text{TIP}$</td>
        <td>$\text{iShares TIPS Bond ETF}$</td>
        <td>$\text{Fixed Income}$</td>
        <td>$\text{US TIPS}$</td>
    </tr>
    <tr>
        <td>$\text{UNG}$</td>
        <td>$\text{United States Natural Gas Fund LP}$</td>
        <td>$\text{Commodities}$</td>
        <td>$\text{Natural Gas}$</td>
    </tr>
    <tr>
        <td>$\text{USO}$</td>
        <td>$\text{United States Oil Fund LP}$</td>
        <td>$\text{Commodities}$</td>
        <td>$\text{Crude Oil}$</td>
    </tr>
    <tr>
        <td>$\text{VEA}$</td>
        <td>$\text{Vanguard Developed Markets Index Fund ETF}$</td>
        <td>$\text{Equities}$</td>
        <td>$\text{DM-Equities}$</td>
    </tr>
    <tr>
        <td>$\text{VNQ}$</td>
        <td>$\text{Vanguard Real Estate ETF}$</td>
        <td>$\text{Real Estate}$</td>
        <td>$\text{Real Estate}$</td>
    </tr>
    <tr>
        <td>$\text{VWO}$</td>
        <td>$\text{Vanguard Emerging Markets Stock Index Fund ETF}$</td>
        <td>$\text{Equities}$</td>
        <td>$\text{EM-Equities}$</td>
    </tr>
    
</table>
<br>

$
\text{
    Our portfolio will consist of the above 20 ETFs, We will perform our momentum strategy analysis
    forall the available dates until January 31, 2022. 
    }
$


$^4 \scriptsize\text{All ETFs are selected from (source).}$

**Documentation**

End of Day US Stock Prices (EOD)
https://data.nasdaq.com/databases/EOD/documentation


$\large\textbf{3 }\quad\large\textbf{Implementation}$

$\textbf{3 - 1. Import functions from util.py}$

In [10]:
from util import *

# let plot display in the notebook instead of in a different window
%matplotlib inline

$\textbf{3 - 2. Get EOD (end of day) data from Quandl}$

In [11]:
secs = ('BIL','BNDX','COW','DBA','EMB',
        'FM','GLD','GOVT','HYG','LQD',
        'MBB','PSP','SCHB','SLV','TIP',
        'UNG','USO','VEA','VNQ','VWO')
start_date,end_date = '2008-01-01','2022-01-31'

data = getQuandlEODData(secs,start_date,end_date,('adj_close','adj_volume'))
data

Quandl | START | Retriving Quandl data for security: ('BIL', 'BNDX', 'COW', 'DBA', 'EMB', 'FM', 'GLD', 'GOVT', 'HYG', 'LQD', 'MBB', 'PSP', 'SCHB', 'SLV', 'TIP', 'UNG', 'USO', 'VEA', 'VNQ', 'VWO')



AttributeError: 'DataFrame' object has no attribute 'get_level_values'

Quandl | START | Retriving Quandl data for security: ('SCHB', 'VEA', 'VWO', 'FM', 'GLD', 'SLV', 'USO', 'UNG', 'DBA', 'COW', 'GOVT', 'TIP', 'MBB', 'LQD', 'HYG', 'BNDX', 'EMB', 'VNQ', 'PSP', 'BIL')

Quandl | DONE  | Returning 3526 dates of data for ('SCHB', 'VEA', 'VWO', 'FM', 'GLD', 'SLV', 'USO', 'UNG', 'DBA', 'COW', 'GOVT', 'TIP', 'MBB', 'LQD', 'HYG', 'BNDX', 'EMB', 'VNQ', 'PSP', 'BIL').



,,adj_close,adj_volume
date,ticker,,
2008-01-02,BIL,85.2938,29450.0000
2008-01-03,BIL,85.2380,30500.0000
2008-01-04,BIL,85.2201,173200.0000
2008-01-07,BIL,85.2936,30300.0000
2008-01-08,BIL,85.2342,18350.0000
...,...,...,...
2021-12-27,BIL,91.4300,730397.0000
2021-12-28,BIL,91.4200,1932997.0000
2021-12-29,BIL,91.4200,1246326.0000
